In [1]:
%load_ext autoreload
%autoreload 2
from mtsccleav import *

In [2]:
from aeon.utils.conversion import convert_collection
from aeon.utils.validation import get_type
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from aeon.transformations.collection.convolution_based import Rocket
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import time


In [3]:
from aeon.classification.convolution_based import (
    Arsenal,
    HydraClassifier,
    MiniRocketClassifier,
    MultiRocketClassifier,
    MultiRocketHydraClassifier,
    RocketClassifier,
)

In [4]:
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score,
    confusion_matrix, matthews_corrcoef
)
import numpy as np

In [5]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeClassifierCV

# Binary

In [6]:
path = "../data/01_single_multi_eq.csv"
include_five_p_cleav=False
include_prob=False
include_ss=True

In [7]:
ts_panel = load_as_pd_multiindex(path)
X = construct_X(ts_panel, include_five_p_cleav=include_five_p_cleav, include_prob=include_prob, include_ss=include_ss)


In [8]:
ts_panel

five_p_cleav_1  five_p_cleav_compl_1  five_p_cleav_2  \
instances time_points                                                         
0         0                        -1                     1               0   
          1                         1                    -1               0   
          2                        -1                     1               0   
          3                         1                    -1               0   
          4                         0                     0               1   
...                               ...                   ...             ...   
826       9                         1                     0               0   
          10                        0                     0              -1   
          11                       -1                     0               0   
          12                        0                     0               1   
          13                       -1                     0               0   

                       five_p_cleav_compl_2  five_p_cleav_prob  \
instances time_points                                            
0         0                               0           0.999468   
          1                               0           0.998997   
          2                               0           0.998971   
          3                               0           0.999318   
          4                              -1           0.999742   
...                                     ...                ...   
826       9                               0           0.000000   
          10                              0           0.000000   
          11                              0           0.000000   
          12                             -1           0.330473   
          13                              1           0.300819   

                       five_p_cleav_ss  five_p_non_cleav_1  \
instances time_points                                        
0         0                          1                   1   
          1                          1                   0   
          2                          1                   0   
          3                          1                  -1   
          4                          1                  -1   
...                                ...                 ...   
826       9                          0                   0   
          10                         0                   0   
          11                         0                   0   
          12                         1                   0   
          13                         1                   1   

                       five_p_non_cleav_compl_1  five_p_non_cleav_2  \
instances time_points                                                 
0         0                                  -1                   0   
          1                                   0                   1   
          2                                  -1                   1   
          3                                   1                   0   
          4                                   1                   0   
...                                         ...                 ...   
826       9                                   0                  -1   
          10                                  0                  -1   
          11                                  0                   1   
          12                                  0                  -1   
          13                                 -1                   0   

                       five_p_non_cleav_compl_2  ...  three_p_cleav_2  \
instances time_points                            ...                    
0         0                                   0  ...                1   
          1                                  -1  ...                0   
          2                                   0  ...                1   
          3                             

In [9]:
# X
# type(X)
# X.isnull().values.any()

In [10]:
X.fillna(-100, inplace=True)
X = pad_multiindex_ts(X,-100)
X_convert=convert_collection(X, "np-list")

In [11]:
# get_type(X), get_type(X_convert)
# np.array(X_convert).shape

In [12]:
y = np.concatenate((np.full((1, 827), 'cleav'), np.full((1, 827), 'non_cleav')), axis=None)
labelencoder = LabelEncoder()
y_transform = labelencoder.fit_transform(y)

In [13]:
def multiclass_specificity(cm):
    FP = cm.sum(axis=0) - np.diag(cm)
    TN = cm.sum() - (FP + cm.sum(axis=1) - np.diag(cm) + np.diag(cm))
    return np.mean(TN / (TN + FP))

In [14]:
def evaluate_classifier(classifier, X, y, n_splits=5, mode="binary"):
    print(f"\n Evaluating {classifier.__class__.__name__}")
    start = time.time()

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    results = {k: [] for k in ["accuracy", "specificity", "sensitivity", "f1", "mcc"]}

    for fold, (train_idx, test_idx) in enumerate(cv.split(X, y), 1):
        # print(f"Fold {fold}")

        X_train = [X[i] for i in train_idx]
        # print(len(train_idx))
        X_test = [X[i] for i in test_idx]
        # print(len(test_idx))
        y_train = y[train_idx]
        y_test = y[test_idx]
    
        clf = classifier
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        mcc = matthews_corrcoef(y_test, y_pred)
        if mode == "binary":
            tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
            accuracy = (tp + tn) / (tp + tn + fp + fn)
            specificity = tn / (tn+fp)
            sensitivity = tp / (tp+fn)
            f1 = 2 * tp / (2 * tp + fp + fn)
            
        elif mode == "multi":
            accuracy = accuracy_score(y_test, y_pred)
            cm = confusion_matrix(y_test, y_pred)
            specificity = multiclass_specificity(cm)
            sensitivity = recall_score(y_test, y_pred, average="macro")
            f1 = f1_score(y_test, y_pred, average="macro")
        else:
            raise ValueError("mode must be either binary or multi.")
    

        results["accuracy"].append(accuracy)
        results["specificity"].append(specificity)
        results["sensitivity"].append(sensitivity)
        results["f1"].append(f1)
        results["mcc"].append(mcc)

    print("\nAverage 5-Fold Results:")
    for metric, scores in results.items():
        print(f"{metric.capitalize():<12}: {np.mean(scores):.4f} ± {np.std(scores):.4f}")
    end = time.time()
    print(f"Execution time: {end - start:.3f} seconds")
    print(f"Average Execution time: {(end - start)/5:.3f} seconds")

In [15]:

evaluate_classifier(RocketClassifier(random_state=18), X_convert, y_transform)
evaluate_classifier(MiniRocketClassifier(random_state=18), X_convert, y_transform)
evaluate_classifier(MultiRocketClassifier(random_state=18), X_convert, y_transform)
evaluate_classifier(HydraClassifier(random_state=18), X_convert, y_transform)
evaluate_classifier(MultiRocketHydraClassifier(random_state=18), X_convert, y_transform)


 Evaluating RocketClassifier

Average 5-Fold Results:
Accuracy    : 0.8585 ± 0.0215
Specificity : 0.8464 ± 0.0136
Sensitivity : 0.8706 ± 0.0396
F1          : 0.8598 ± 0.0243
Mcc         : 0.7178 ± 0.0430
Execution time: 20.131 seconds
Average Execution time: 4.026 seconds

 Evaluating MiniRocketClassifier

Average 5-Fold Results:
Accuracy    : 0.8301 ± 0.0144
Specificity : 0.8210 ± 0.0182
Sensitivity : 0.8392 ± 0.0269
F1          : 0.8315 ± 0.0157
Mcc         : 0.6608 ± 0.0291
Execution time: 4.085 seconds
Average Execution time: 0.817 seconds

 Evaluating MultiRocketClassifier

Average 5-Fold Results:
Accuracy    : 0.8162 ± 0.0061
Specificity : 0.8005 ± 0.0146
Sensitivity : 0.8319 ± 0.0260
F1          : 0.8189 ± 0.0096
Mcc         : 0.6332 ± 0.0128
Execution time: 22.157 seconds
Average Execution time: 4.431 seconds

 Evaluating HydraClassifier

Average 5-Fold Results:
Accuracy    : 0.8428 ± 0.0175
Specificity : 0.8053 ± 0.0178
Sensitivity : 0.8803 ± 0.0202
F1          : 0.8485 ± 0.0

# Multi

In [16]:
ts_panel = load_as_pd_multiindex(path)
X_5p = construct_X(ts_panel, include_five_p_cleav=True, include_prob=include_prob, include_ss=include_ss)
X_5p.columns = X_5p.columns.str.replace("five_p_", "", regex=False)
X_3p = construct_X(ts_panel, include_five_p_cleav=False, include_prob=include_prob, include_ss=include_ss)
X_3p .columns = X_3p.columns.str.replace("three_p_", "", regex=False)

In [17]:
X_5p.fillna(-100, inplace=True)
X_5p = pad_multiindex_ts(X_5p,-100)
X_5p_convert=convert_collection(X_5p, "np-list")
X_3p.fillna(-100, inplace=True)
X_3p = pad_multiindex_ts(X_3p,-100)
X_3p_convert=convert_collection(X_3p, "np-list")


In [18]:
X_convert = np.concatenate((X_5p_convert, X_3p_convert), axis=0)
# np.array(X_convert).shape

In [19]:
y = np.concatenate((np.full((1, 827), 'five_p_cleav'), np.full((1, 827), 'non_cleav'), np.full((1, 827), 'three_p_cleav'), np.full((1, 827), 'non_cleav')), axis=None)
labelencoder = LabelEncoder()
y_transform = labelencoder.fit_transform(y)
# print(np.unique(y_transform))
# print(len(np.unique(y_transform)))

In [20]:
evaluate_classifier(RocketClassifier(random_state=18), X_convert, y_transform, mode = "multi")
evaluate_classifier(MiniRocketClassifier(random_state=18), X_convert, y_transform, mode = "multi")
evaluate_classifier(MultiRocketClassifier(random_state=18), X_convert, y_transform, mode = "multi")
evaluate_classifier(HydraClassifier(random_state=18), X_convert, y_transform, mode = "multi")
evaluate_classifier(MultiRocketHydraClassifier(random_state=18), X_convert, y_transform, mode = "multi")


 Evaluating RocketClassifier

Average 5-Fold Results:
Accuracy    : 0.8446 ± 0.0105
Specificity : 0.9117 ± 0.0064
Sensitivity : 0.8365 ± 0.0126
F1          : 0.8422 ± 0.0112
Mcc         : 0.7498 ± 0.0174
Execution time: 58.455 seconds
Average Execution time: 11.691 seconds

 Evaluating MiniRocketClassifier

Average 5-Fold Results:
Accuracy    : 0.8289 ± 0.0033
Specificity : 0.9033 ± 0.0037
Sensitivity : 0.8170 ± 0.0085
F1          : 0.8235 ± 0.0043
Mcc         : 0.7246 ± 0.0061
Execution time: 22.751 seconds
Average Execution time: 4.550 seconds

 Evaluating MultiRocketClassifier

Average 5-Fold Results:
Accuracy    : 0.7681 ± 0.0178
Specificity : 0.8726 ± 0.0114
Sensitivity : 0.7612 ± 0.0213
F1          : 0.7621 ± 0.0187
Mcc         : 0.6285 ± 0.0304
Execution time: 66.465 seconds
Average Execution time: 13.293 seconds

 Evaluating HydraClassifier

Average 5-Fold Results:
Accuracy    : 0.8325 ± 0.0106
Specificity : 0.9018 ± 0.0071
Sensitivity : 0.8137 ± 0.0142
F1          : 0.8271 ± 

# End

In [21]:
import datetime
print(f"This Notebook last end-to-end runs on: {datetime.datetime.now()}\n")

This Notebook last end-to-end runs on: 2025-07-07 17:17:01.914679

